<a href="https://colab.research.google.com/github/otamajakusi/wav2vec2/blob/main/wav2vec2_ja.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [30]:
%%capture
!pip install datasets==1.13.3
!pip install transformers==4.11.3
!pip install torchaudio
!pip install librosa
!pip jiwer

In [2]:
%%capture
# If you don't have jsut package, download and extract it.
# https://sites.google.com/site/shinnosuketakamichi/publication/jsut
# !wget http://ss-takashi.sakura.ne.jp/corpus/jsut_ver1.1.zip
# !unzip jsut_ver1.1.zip -d /content/drive/MyDrive/

In [3]:
!git clone https://github.com/sarulab-speech/jsut-label

Cloning into 'jsut-label'...
remote: Enumerating objects: 9628, done.
remote: Counting objects: 100% (9628/9628), done.
remote: Compressing objects: 100% (8841/8841), done.
remote: Total 9628 (delta 4598), reused 5805 (delta 784), pack-reused 0
Receiving objects: 100% (9628/9628), 8.61 MiB | 13.03 MiB/s, done.
Resolving deltas: 100% (4598/4598), done.


In [41]:
import yaml
from pathlib import Path
from datasets import Dataset

def create_dataset(yaml_file, wav_dir: Path):
    paths = []
    texts = []

    with open(yaml_file) as f:
        meta = yaml.safe_load(f)

    for i, m in enumerate(meta.keys()):
        wav = wav_dir / m
        paths.append(f"{wav}.wav")

        sentence = meta[m]["phone_level3"]
        sentence = sentence.replace("-pau", " ")
        sentence = sentence.replace("-", "")
        sentence = sentence.lower() + " "
        texts.append(sentence)
    data = Dataset.from_dict({"path": paths, "text": texts})
    data = data.shuffle(seed=42)
    return data.train_test_split(test_size=0.1, seed=42)

In [42]:
dataset = create_dataset("/content/jsut-label/text_kana/basic5000.yaml", Path("/content/drive/MyDrive/jsut_ver1.1/basic5000/wav"))
dataset

DatasetDict({
    train: Dataset({
        features: ['path', 'text'],
        num_rows: 4500
    })
    test: Dataset({
        features: ['path', 'text'],
        num_rows: 500
    })
})

In [43]:
train_dataset = dataset["train"]
test_dataset = dataset["test"]

In [44]:
def extract_all_chars(batch):
    all_text = " ".join(batch["text"])
    vocab = list(set(all_text))
    return {"vocab": [vocab], "all_text": [all_text]}

vocab_train = train_dataset.map(
    extract_all_chars,
    batched=True,
    batch_size=-1,
    keep_in_memory=True,
    remove_columns=train_dataset.column_names,
)
vocab_test = test_dataset.map(
    extract_all_chars,
    batched=True,
    batch_size=-1,
    keep_in_memory=True,
    remove_columns=test_dataset.column_names,
)

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [45]:
vocab_list = list(set(vocab_train["vocab"][0]) | set(vocab_test["vocab"][0]))
vocab_dict = {v: k for k, v in enumerate(vocab_list)}
vocab_dict["|"] = vocab_dict[" "]
del vocab_dict[" "]
vocab_dict["[UNK]"] = len(vocab_dict)
vocab_dict["[PAD]"] = len(vocab_dict)
vocab_dict

{'[PAD]': 26,
 '[UNK]': 25,
 'a': 3,
 'b': 5,
 'c': 13,
 'd': 7,
 'e': 24,
 'f': 4,
 'g': 21,
 'h': 15,
 'i': 1,
 'j': 10,
 'k': 2,
 'l': 17,
 'm': 19,
 'n': 23,
 'o': 8,
 'p': 6,
 'r': 12,
 's': 22,
 't': 11,
 'u': 16,
 'v': 14,
 'w': 0,
 'y': 9,
 'z': 18,
 '|': 20}

In [46]:
import json

with open("vocab.json", "w") as vocab_file:
    json.dump(vocab_dict, vocab_file)

In [47]:
from transformers import Wav2Vec2CTCTokenizer
from transformers import Wav2Vec2FeatureExtractor
from transformers import Wav2Vec2Processor

tokenizer = Wav2Vec2CTCTokenizer(
    "./vocab.json", unk_token="[UNK]", pad_token="[PAD]", word_delimiter_token="|"
)

feature_extractor = Wav2Vec2FeatureExtractor(
    feature_size=1,
    sampling_rate=16000,
    padding_value=0.0,
    do_normalize=True,
    return_attention_mask=True,
)

processor = Wav2Vec2Processor(feature_extractor=feature_extractor, tokenizer=tokenizer)

In [48]:
processor.save_pretrained("/content/drive/MyDrive/container_0/wav2vec2-large-xlsr-ja")

Configuration saved in /content/drive/MyDrive/container_0/wav2vec2-large-xlsr-ja/preprocessor_config.json
tokenizer config file saved in /content/drive/MyDrive/container_0/wav2vec2-large-xlsr-ja/tokenizer_config.json
Special tokens file saved in /content/drive/MyDrive/container_0/wav2vec2-large-xlsr-ja/special_tokens_map.json


In [49]:
import torchaudio
import librosa
import numpy as np

def speech_file_to_array(batch):
    speech_array, sampling_rate = torchaudio.load(batch["path"])
    speech = speech_array[0].numpy()
    batch["speech"] = librosa.resample(np.asarray(speech), sampling_rate, 16_000)
    batch["sampling_rate"] = 16_000
    batch["target_text"] = batch["text"]
    return batch

train_dataset = train_dataset.map(
    speech_file_to_array, remove_columns=train_dataset.column_names
)
test_dataset = test_dataset.map(
    speech_file_to_array, remove_columns=test_dataset.column_names
)

  0%|          | 0/4500 [00:00<?, ?ex/s]

  0%|          | 0/500 [00:00<?, ?ex/s]

In [50]:
train_dataset[:10]['sampling_rate']

[16000, 16000, 16000, 16000, 16000, 16000, 16000, 16000, 16000, 16000]

In [54]:
import IPython.display as ipd
import numpy as np
import random

rand_int = random.randint(0, len(train_dataset))

ipd.Audio(data=np.asarray(train_dataset[rand_int]["speech"]), autoplay=True, rate=16000)

In [55]:
def prepare_dataset(batch):
    # check that all files have the correct sampling rate
    assert (
        len(set(batch["sampling_rate"])) == 1
    ), f"Make sure all inputs have the same sampling rate of {processor.feature_extractor.sampling_rate}."

    batch["input_values"] = processor(
        batch["speech"], sampling_rate=batch["sampling_rate"][0]
    ).input_values

    with processor.as_target_processor():
        batch["labels"] = processor(batch["target_text"]).input_ids
    return batch

train_dataset = train_dataset.map(
    prepare_dataset,
    remove_columns=train_dataset.column_names,
    batch_size=4,
    batched=True,
)
test_dataset = test_dataset.map(
    prepare_dataset,
    remove_columns=test_dataset.column_names,
    batch_size=4,
    batched=True,
)

  0%|          | 0/1125 [00:00<?, ?ba/s]

/usr/local/lib/python3.7/dist-packages/numpy/core/_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)


  0%|          | 0/125 [00:00<?, ?ba/s]

In [56]:
import torch

from dataclasses import dataclass, field
from typing import Any, Dict, List, Optional, Union

from transformers import Wav2Vec2Processor


@dataclass
class DataCollatorCTCWithPadding:
    """
    Data collator that will dynamically pad the inputs received.
    Args:
        processor (:class:`~transformers.Wav2Vec2Processor`)
            The processor used for proccessing the data.
        padding (:obj:`bool`, :obj:`str` or :class:`~transformers.tokenization_utils_base.PaddingStrategy`, `optional`, defaults to :obj:`True`):
            Select a strategy to pad the returned sequences (according to the model's padding side and padding index)
            among:
            * :obj:`True` or :obj:`'longest'`: Pad to the longest sequence in the batch (or no padding if only a single
              sequence if provided).
            * :obj:`'max_length'`: Pad to a maximum length specified with the argument :obj:`max_length` or to the
              maximum acceptable input length for the model if that argument is not provided.
            * :obj:`False` or :obj:`'do_not_pad'` (default): No padding (i.e., can output a batch with sequences of
              different lengths).
        max_length (:obj:`int`, `optional`):
            Maximum length of the ``input_values`` of the returned list and optionally padding length (see above).
        max_length_labels (:obj:`int`, `optional`):
            Maximum length of the ``labels`` returned list and optionally padding length (see above).
        pad_to_multiple_of (:obj:`int`, `optional`):
            If set will pad the sequence to a multiple of the provided value.
            This is especially useful to enable the use of Tensor Cores on NVIDIA hardware with compute capability >=
            7.5 (Volta).
    """

    processor: Wav2Vec2Processor
    padding: Union[bool, str] = True
    max_length: Optional[int] = None
    max_length_labels: Optional[int] = None
    pad_to_multiple_of: Optional[int] = None
    pad_to_multiple_of_labels: Optional[int] = None

    def __call__(
        self, features: List[Dict[str, Union[List[int], torch.Tensor]]]
    ) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lenghts and need
        # different padding methods
        input_features = [
            {"input_values": feature["input_values"]} for feature in features
        ]
        label_features = [{"input_ids": feature["labels"]} for feature in features]

        batch = self.processor.pad(
            input_features,
            padding=self.padding,
            max_length=self.max_length,
            pad_to_multiple_of=self.pad_to_multiple_of,
            return_tensors="pt",
        )
        with self.processor.as_target_processor():
            labels_batch = self.processor.pad(
                label_features,
                padding=self.padding,
                max_length=self.max_length_labels,
                pad_to_multiple_of=self.pad_to_multiple_of_labels,
                return_tensors="pt",
            )

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(
            labels_batch.attention_mask.ne(1), -100
        )

        batch["labels"] = labels

        return batch

In [57]:
data_collator = DataCollatorCTCWithPadding(processor=processor, padding=True)

In [58]:
!pip install jiwer

In [59]:
from datasets import load_metric

def compute_metrics(pred):
    pred_logits = pred.predictions
    pred_ids = np.argmax(pred_logits, axis=-1)

    pred.label_ids[pred.label_ids == -100] = processor.tokenizer.pad_token_id

    pred_str = processor.batch_decode(pred_ids)
    # we do not want to group tokens when computing the metrics
    label_str = processor.batch_decode(pred.label_ids, group_tokens=False)

    wer = wer_metric.compute(predictions=pred_str, references=label_str)

    return {"wer": wer}

wer_metric = load_metric("wer")

In [60]:
from transformers import Wav2Vec2ForCTC
from transformers import TrainingArguments, Trainer

model = Wav2Vec2ForCTC.from_pretrained(
    "facebook/wav2vec2-large-xlsr-53",
    attention_dropout=0.1,
    hidden_dropout=0.1,
    feat_proj_dropout=0.0,
    mask_time_prob=0.05,
    layerdrop=0.1,
    gradient_checkpointing=True,
    ctc_loss_reduction="mean",
    pad_token_id=processor.tokenizer.pad_token_id,
    vocab_size=len(processor.tokenizer),
)

model.freeze_feature_extractor()

training_args = TrainingArguments(
    output_dir="/content/drive/MyDrive/container_0/ckpts/",
    logging_dir="/content/drive/MyDrive/container_0/runs/",
    group_by_length=True,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    gradient_accumulation_steps=2,
    evaluation_strategy="steps",
    num_train_epochs=30,
    fp16=True,
    save_steps=200,
    eval_steps=200,
    logging_steps=200,
    learning_rate=4e-4,
    warmup_steps=int(0.1 * 1320),  # 10%
    save_total_limit=2,
)

trainer = Trainer(
    model=model,
    data_collator=data_collator,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    tokenizer=processor.feature_extractor,
)

loading configuration file https://huggingface.co/facebook/wav2vec2-large-xlsr-53/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/8508c73cd595eb416a1d517b90762416c0bc6cfbef529578079aeae4d8c14336.7581ed2ee0c677f1e933180df51bd1a668c4a2b6d5fd1297d32069373dac097c
Model config Wav2Vec2Config {
  "activation_dropout": 0.0,
  "apply_spec_augment": true,
  "architectures": [
    "Wav2Vec2ForPreTraining"
  ],
  "attention_dropout": 0.1,
  "bos_token_id": 1,
  "classifier_proj_size": 256,
  "codevector_dim": 768,
  "contrastive_logits_temperature": 0.1,
  "conv_bias": true,
  "conv_dim": [
    512,
    512,
    512,
    512,
    512,
    512,
    512
  ],
  "conv_kernel": [
    10,
    3,
    3,
    3,
    3,
    2,
    2
  ],
  "conv_stride": [
    5,
    2,
    2,
    2,
    2,
    2,
    2
  ],
  "ctc_loss_reduction": "mean",
  "ctc_zero_infinity": false,
  "diversity_loss_weight": 0.1,
  "do_stable_layer_norm": true,
  "eos_token_id": 2,
  "feat_extract_activatio

In [ ]:
trainer.train()

***** Running training *****
  Num examples = 4500
  Num Epochs = 30
  Instantaneous batch size per device = 4
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 2
  Total optimization steps = 16860
/usr/local/lib/python3.7/dist-packages/transformers/models/wav2vec2/modeling_wav2vec2.py:882: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  return (input_length - kernel_size) // stride + 1


Step,Training Loss,Validation Loss


In [ ]:
trainer.save_model("/content/drive/MyDrive/container_0/wav2vec2-large-xlsr-ja")